In [1]:
import warnings
warnings.filterwarnings('ignore')
import stwcs
import glob
import sys
import os
import shutil
import time
import filecmp
import astroquery
import progressbar
import copy
import requests
import random
import astropy.wcs as wcs
import numpy as np
from contextlib import contextmanager
from astropy import units as u
from astropy.utils.data import clear_download_cache,download_file
from astropy.io import fits
from astropy.table import Table, Column, unique
from astropy.time import Time
from astroscrappy import detect_cosmics
from stwcs import updatewcs
from scipy.interpolate import interp1d

# Internal dependencies
from common import Constants
from common import Options
from common import Settings
from common import Util
from nbutils import get_filter, get_instrument, get_chip, get_filter, input_list
from nbutils import get_zpt, add_visit_info, organize_reduction_tables, pick_deepest_images

@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

with suppress_stdout():
    from drizzlepac import tweakreg,astrodrizzle,catalogs,photeq
    from astroquery.mast import Observations
    from astropy.coordinates import SkyCoord

In [3]:
import glob,os
import jwst
from astropy.io import fits
print(f'JWST version: {jwst.__version__}')

root_dir = '.'
filter_name = 'F115W'
inputfiles_level3_filter = glob.glob('jwstred/jhat/*')

JWST version: 1.14.0


In [4]:
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

In [5]:
input_images = glob.glob('jwstred_temp+mosaic/*fits')
table = input_list(input_images)
tables = organize_reduction_tables(table, byvisit=False)

In [6]:
filter_name = table[0]['filter']

In [7]:
nircam_asn_file = f'jwstred_temp+mosaic/{filter_name}.json'
base_filenames = np.array([os.path.basename(r['image']) for r in table])
asn3 = asn_from_list.asn_from_list(base_filenames, 
    rule=DMS_Level3_Base, product_name=filter_name)

In [8]:
with open(nircam_asn_file, 'w') as outfile:
    name, serialized = asn3.dump(format='json')
    outfile.write(serialized)

In [9]:
print(asn3)

jwnoprogram-a3001_none_00003_asn with 1 products
Rule=DMS_Level3_Base
No constraints
Products:
	f115w with 16 members


In [10]:
from jwst.pipeline import calwebb_image3
image3 = calwebb_image3.Image3Pipeline()

outdir_level3 = os.path.join('jwstred_temp+mosaic/', 'out')
if not os.path.exists(outdir_level3):
    os.makedirs(outdir_level3)

image3.output_dir = outdir_level3
image3.save_results = True
image3.tweakreg.skip = True
image3.skymatch.skip = True
image3.skymatch.match_down = False
image3.source_catalog.skip=False

image3.run(nircam_asn_file)

2024-09-10 09:13:08,245 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2024-09-10 09:13:08,247 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-09-10 09:13:08,253 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2024-09-10 09:13:08,257 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2024-09-10 09:13:08,262 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2024-09-10 09:13:08,267 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2024-09-10 09:13:08,272 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.
2024-09-10 09:13:08,374 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline running with args ('jwstred_temp+mosaic/f115w.json',).
2024-09-10 09:13:08,400 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline parameters are:
  pre_hooks: []
  post_hooks: []
  output_file: None
  output_dir: j

2024-09-10 09:13:43,283 - stpipe.Image3Pipeline.outlier_detection - INFO - Saved model in jw02784002001_02101_00001_nrcb4_outlier_i2d.fits
2024-09-10 09:13:43,370 - stpipe.Image3Pipeline.outlier_detection - INFO - 4 exposures to drizzle together
2024-09-10 09:13:47,490 - stpipe.Image3Pipeline.outlier_detection - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:13:51,809 - stpipe.Image3Pipeline.outlier_detection - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:13:56,462 - stpipe.Image3Pipeline.outlier_detection - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:14:00,572 - stpipe.Image3Pipeline.outlier_detection - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:14:02,193 - stpipe.Image3Pipeline.outlier_detection - INFO - Sleeping for 1.0s and reading back saved data
2024-09-10 09:14:03,414 - stpipe.Image3Pipeline.outlier_detection - INFO - Shape of read back model data matches that of input model
2024-09-10 09:14:03,415 - stpipe.Image3P

2024-09-10 09:17:23,963 - stpipe.Image3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 354 (0.01%)
2024-09-10 09:17:24,685 - stpipe.Image3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 371 (0.01%)
2024-09-10 09:17:25,506 - stpipe.Image3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 264 (0.01%)
2024-09-10 09:17:26,458 - stpipe.Image3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 362 (0.01%)
2024-09-10 09:17:27,290 - stpipe.Image3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 284 (0.01%)
2024-09-10 09:17:28,041 - stpipe.Image3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 259 (0.01%)
2024-09-10 09:17:28,857 - stpipe.Image3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 451 (0.01%)
2024-09-10 09:17:29,644 - stpipe.Image3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 303 (0.01%)
2024-09-10 09:17:30,445 - stpipe.Image3Pipeline.

2024-09-10 09:19:45,801 - stpipe.Image3Pipeline.resample - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:19:49,819 - stpipe.Image3Pipeline.resample - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:19:54,493 - stpipe.Image3Pipeline.resample - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:19:59,898 - stpipe.Image3Pipeline.resample - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:20:04,392 - stpipe.Image3Pipeline.resample - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:20:05,901 - stpipe.Image3Pipeline.resample - INFO - Resampling var_poisson
2024-09-10 09:20:08,896 - stpipe.Image3Pipeline.resample - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:20:13,097 - stpipe.Image3Pipeline.resample - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:20:17,250 - stpipe.Image3Pipeline.resample - INFO - Drizzling (2048, 2048) --> (4245, 4265)
2024-09-10 09:20:21,606 - stpipe.Image3Pipeline.resample - INFO

In [3]:
nirc_asn_file = 'jwstred/jhat/asnfile.json'
with open(nirc_asn_file, 'w') as outfile:
    name, serialized = asn3.dump(format='json')
    outfile.write(serialized)
print(asn3)

2024-08-08 19:25:02,777 - stpipe - WARNING - /home/aswin/anaconda3/envs/webbphot/lib/python3.10/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)



jwnoprogram-a3001_none_00002_asn with 1 products
Rule=DMS_Level3_Base
No constraints
Products:
	F115W with 17 members


In [4]:
from jwst.pipeline import calwebb_image3
image3 = calwebb_image3.Image3Pipeline()

2024-08-08 19:25:05,135 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2024-08-08 19:25:05,137 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-08-08 19:25:05,139 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2024-08-08 19:25:05,142 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2024-08-08 19:25:05,144 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2024-08-08 19:25:05,147 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2024-08-08 19:25:05,149 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.


In [5]:
outdir_level3 = 'jwstred/out'

In [6]:
image3.output_dir = outdir_level3
image3.save_results = True
### IMPORTANT!! you have to set image3.tweakreg.skip = True, otherwise tweakreg will screw up the WCS again!
image3.tweakreg.skip = True
#image3.tweakreg.align_to_gaia = True
image3.skymatch.skip = True
#image3.skymatch.skymethod = 'global+match'
image3.skymatch.match_down = False
image3.source_catalog.skip=False
#image3.skymatch.subtract = True

In [8]:
image3.run(nirc_asn_file)

2024-08-08 19:25:49,233 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline running with args ('jwstred/jhat/asnfile.json',).
2024-08-08 19:25:49,247 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline parameters are:
  pre_hooks: []
  post_hooks: []
  output_file: None
  output_dir: jwstred/out
  output_ext: .fits
  output_use_model: False
  output_use_index: True
  save_results: True
  skip: False
  suffix: image3pipeline
  search_output_file: True
  input_dir: jwstred/jhat
  steps:
    assign_mtwcs:
      pre_hooks: []
      post_hooks: []
      output_file: None
      output_dir: None
      output_ext: .fits
      output_use_model: True
      output_use_index: True
      save_results: False
      skip: False
      suffix: assign_mtwcs
      search_output_file: True
      input_dir: jwstred/jhat
    tweakreg:
      pre_hooks: []
      post_hooks: []
      output_file: None
      output_dir: None
      output_ext: .fits
      output_use_model: True
      output_use_index: True
    

AssertionError: 